# 3D mesh tallies

This example makes a simple geometry from a few different materials and plots a 3D regular mesh tally of the results.

A mesh tally allows a visual inspection of the neutron behaviour spatially throughout the geometry.

The geometry is subdivided into many rectangles and the neutron behaviour is recorded (tallied) by the simulation in each of the small rectangles.

The resulting vtk can be viewed to see the 3D tally.

In [ ]:
import openmc
import os

# MATERIALS

breeder_material = openmc.Material(1, "PbLi")   # Pb84.2Li15.8
breeder_material.add_element('Pb', 84.2, percent_type='ao')
breeder_material.add_element('Li', 15.8, percent_type='ao', enrichment=7.0, enrichment_target='Li6', enrichment_type='ao')   # natural enrichment = 7% Li6
breeder_material.set_density('atom/b-cm', 3.2720171e-2)   # around 11 g/cm3

copper = openmc.Material(name='Copper')
copper.set_density('g/cm3', 8.5)
copper.add_element('Cu', 1.0)

eurofer = openmc.Material(name='iron')
eurofer.set_density('g/cm3', 7.75)
eurofer.add_element('Fe', 89.067, percent_type='wo')

mats = openmc.Materials([breeder_material, eurofer, copper])


# GEOMETRY

# surfaces
central_sol_surface = openmc.ZCylinder(r=100)
central_shield_outer_surface = openmc.ZCylinder(r=110)
vessel_inner_surface = openmc.Sphere(r=500)
first_wall_outer_surface = openmc.Sphere(r=510)
breeder_blanket_outer_surface = openmc.Sphere(r=610, boundary_type='vacuum')

# cells
central_sol_region = -central_sol_surface & -breeder_blanket_outer_surface
central_sol_cell = openmc.Cell(region=central_sol_region)
central_sol_cell.fill = copper

central_shield_region = +central_sol_surface & -central_shield_outer_surface & -breeder_blanket_outer_surface
central_shield_cell = openmc.Cell(region=central_shield_region)
central_shield_cell.fill = eurofer

inner_vessel_region = -vessel_inner_surface & +central_shield_outer_surface
inner_vessel_cell = openmc.Cell(region=inner_vessel_region)
# no material set as default is vacuum

first_wall_region = -first_wall_outer_surface & +vessel_inner_surface
first_wall_cell = openmc.Cell(region=first_wall_region)
first_wall_cell.fill = eurofer

breeder_blanket_region = +first_wall_outer_surface & -breeder_blanket_outer_surface & +central_shield_outer_surface
breeder_blanket_cell = openmc.Cell(region=breeder_blanket_region)
breeder_blanket_cell.fill = breeder_material

universe = openmc.Universe(cells=[central_sol_cell, central_shield_cell, inner_vessel_cell, first_wall_cell, breeder_blanket_cell])
geom = openmc.Geometry(universe)


# SIMULATION SETTINGS

# Instantiate a Settings object
sett = openmc.Settings()
batches = 2
sett.batches = batches
sett.inactive = 0
sett.particles = 5000
sett.run_mode = 'fixed source'

# Create a DT point source
source = openmc.Source()
source.angle = openmc.stats.Isotropic()
source.energy = openmc.stats.Discrete([14e6], [1])
source.space = openmc.stats.Point((150, 150, 0))

sett.source = source

This section creates a 3D regular mesh from one coordinate to another coordinate with a specificed resolution in each axis.

In [ ]:
mesh = openmc.RegularMesh(mesh_id=1)  # note the mesh_id is set to 1, this is needed later

mesh.dimension = [100, 50, 100]  # width, depth, height

mesh.lower_left = [-750, 0, -750]  # x,y,z coordinates
mesh.upper_right = [750, 750, 750]  # x,y,z coordinates




This section creates two tallies on the previously created mesh and records heating and tritium production

In [ ]:
tallies = openmc.Tallies()
# Create mesh filter for tally
mesh_filter = openmc.MeshFilter(mesh)

# Create flux mesh tally to score flux
mesh_tally = openmc.Tally(tally_id=1, name='tbr_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['(n,Xt)']
tallies.append(mesh_tally)

# Create flux mesh tally to score flux
mesh_tally = openmc.Tally(tally_id=2, name='heating_on_mesh')  # note the tally_id is specified
mesh_tally.filters = [mesh_filter]
mesh_tally.scores = ['heating']
tallies.append(mesh_tally)


This performs the simulation

In [ ]:
# deletes old statepoint and sumary files
!rm s*.h5

# Run OpenMC!
model = openmc.model.Model(geom, mats, sett, tallies)
sp_filename = model.run()

This command runs a python function that is able to extract mesh tally data from the statepoint.h5 file and save the mesh tally as a vtk file

In [ ]:
from statepoint_to_vtk import initiate_mesh

initiate_mesh(
    statepoint_filename=sp_filename,
    tally_name='tbr_on_mesh',
    output_filename='tbr_tally_on_mesh.vtk',
    mesh_id=1,  # note the mesh_id from earlier is used
    tally_id=1  # note the tally_id from earlier is used
)

initiate_mesh(
    statepoint_filename=sp_filename,
    tally_name='heating_on_mesh',
    output_filename='heating_tally_on_mesh.vtk',
    mesh_id=1,  # note the mesh_id from earlier is used
    tally_id=2 # note the tally_id from earlier is used
)

This next command generates a download link to the 3d vtk file generated.

Click on the link to download the vtk file onto your base computer and open it with  a VTK file reader such as Paraview or Visit

Paraview can be downloaded here https://www.paraview.org/download/
Visit can be downloaded here https://wci.llnl.gov/simulation/computer-codes/visit/downloads

In [ ]:
from IPython.display import FileLink
display(FileLink('heating_tally_on_mesh.vtk'))
display(FileLink('tbr_tally_on_mesh.vtk'))